In [1]:
%%time
import pandas as pd
import numpy as np
import os
import glob
import easyocr
import cv2
from joblib import Parallel, delayed
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests
from multiprocessing import Pool
import warnings
import concurrent.futures

%matplotlib inline

warnings.filterwarnings('ignore')

CPU times: user 3.61 s, sys: 584 ms, total: 4.2 s
Wall time: 7.76 s


In [2]:
input_path = '/kaggle/input/ml-dataset/'
test = pd.read_csv(input_path+'test.csv')
display(test)

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


# get text

In [3]:
reader = easyocr.Reader(['en', 'es'])

In [4]:
def url_image(link):
    try:
        r = requests.get(link)
        if r.status_code == 200:
#             print("Image content downloaded.")
            return r.content
        else:
            print(f"Error downloading image, status code: {r.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

def cvrreturner(link):
    returnval = url_image(link)
    if returnval is not None:
        # Convert the binary image data into a format OpenCV can work with
        image = cv2.imdecode(np.frombuffer(returnval, np.uint8), cv2.IMREAD_COLOR)
        if image is not None:
            # Convert to RGB format (OpenCV loads images as BGR by default)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Pass image directly to EasyOCR for text recognition
            ocr_text = reader.readtext(image)
            # Join all detected text into a single sentence
            sentence = " ".join([o[1] for o in ocr_text])
            return sentence
        else:
            print("Error decoding the image")
            return None
    else:
        return None

In [5]:
value = cvrreturner(test['image_link'][0])
value

'S6ban 36in - ?ug 91 bauMn I5eii'

# Extract all the ocr text and save

In [6]:
def main_extractor_for_df(df):
    linklist = list(df['image_link'])
    textlist = []
    for link in tqdm(linklist):
        value = cvrreturner(link)
        textlist.append(value)
    df['textfromimg'] = textlist
    return df

In [7]:
test40 = test.head(40)

df_chunks = np.array_split(test40, 4)  

for i, chunk in enumerate(df_chunks):
    print(f"Chunk {i+1} has {len(chunk)} rows")

df1 = df_chunks[0] 
df2 = df_chunks[1]  
df3 = df_chunks[2] 
df4 = df_chunks[3]  

Chunk 1 has 10 rows
Chunk 2 has 10 rows
Chunk 3 has 10 rows
Chunk 4 has 10 rows


In [8]:
def run_functions_in_parallel():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(main_extractor_for_df(df1)),
            executor.submit(main_extractor_for_df(df2)),
            executor.submit(main_extractor_for_df(df3)),
            executor.submit(main_extractor_for_df(df4)),
        ]
        concurrent.futures.wait(futures)
run_functions_in_parallel()

100%|██████████| 10/10 [00:01<00:00,  8.38it/s]


In [9]:
# df1.to_csv('file1.csv')
# df1.to_csv('file2.csv')
# df1.to_csv('file3.csv')
# df1.to_csv('file4.csv')

# control dataset 

In [10]:
subset_df = test.iloc[12001:24000]  
print('===============================================================================================')
print(subset_df.index)
print('===============================================================================================')

RangeIndex(start=12001, stop=24000, step=1)


In [11]:
targetdf = subset_df.copy()

df_chunks = np.array_split(targetdf, 4)  

for i, chunk in enumerate(df_chunks):
    print(f"Chunk {i+1} has {len(chunk)} rows")

df1 = df_chunks[0] 
df2 = df_chunks[1]  
df3 = df_chunks[2] 
df4 = df_chunks[3]  

Chunk 1 has 3000 rows
Chunk 2 has 3000 rows
Chunk 3 has 3000 rows
Chunk 4 has 2999 rows


In [12]:
def run_functions_in_parallel():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(main_extractor_for_df(df1)),
            executor.submit(main_extractor_for_df(df2)),
            executor.submit(main_extractor_for_df(df3)),
            executor.submit(main_extractor_for_df(df4)),
        ]
        concurrent.futures.wait(futures)
run_functions_in_parallel()

100%|██████████| 2999/2999 [24:26<00:00,  2.04it/s]


In [13]:
df1.to_csv('12001-24000-1.csv')
df2.to_csv('12001-24000-2.csv')
df3.to_csv('12001-24000-3.csv')
df4.to_csv('12001-24000-4.csv')